# Customizing Templates and Components

**Why Customize?**

While Deeplay provides a robust set of pre-defined templates and components, there may be cases where you'd like to customize these to better suit your specific problem. Customization can range from minor changes to existing templates to creating entirely new components.



### Customization using Config

**Decoding Automatic Summaries**

When you print your model using `print(model)`, `deeplay` will provide an automatic summary that shows the hierarchical structure of the model. Your `Config` object should mirror this hierarchy, allowing you to easily set or modify specific attributes and layers within the model. For example, let's look at the initial lines of the `ImageClassifier` summary:

```python
ImageClassifier(
  (num_classes): 10
  (backbone): ImageToVectorEncoder(
    (depth): 4
    (input_block): Template(
      (layer): LazyConv2d(0, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (activation): ReLU()
    )
    (encoder_blocks): ModuleList(
      (0): Template(
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (layer): LazyConv2d(0, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): ReLU()
      )
    ... continues
```

Pay attention to the names within parentheses — `backbone`, `depth`, `input_block`, `encoder_blocks`, `0`, `layer` and more. To change the `kernel_size` attribute, you can create a `Config` object that mimics this structure:

In [6]:
from deeplay import Config 

config = Config().backbone.encoder_blocks[0].layer.kernel_size(5)

**Note**: if you want to update many blocks at once, you can use slicing:

In [7]:
config = Config().backbone.encoder_blocks[0:4].layer.kernel_size(5)

**Note**: If you want to update all blocks at once, you can omit the indexing:

In [8]:
config = Config().backbone.encoder_blocks.layer.kernel_size(5)

**Note**: Just like you can update attributes of the layers, you can also change the type of layers themselves. For instance, to change the activation function to `LeakyReLU` for all blocks, you can do:

In [9]:
import torch.nn as nn

config = Config().backbone.encoder_blocks.activation(nn.LeakyReLU, negative_slope=0.1)

This allows you to easily swap out modules and tune the model to better fit your specific needs.

### 2. Customization using Templates

**Switching to a More Complex Block Design**

Let's continue from our previous example. Deeplay allows you to define the structure of your blocks via templates. A template specifies the sequence and type of layers you want to stack together in a block. Here's how you can define a new template to include a batch normalization layer in each block:

This level of customization opens up a wide array of possibilities, allowing you to quickly experiment with various architectures while keeping the rest of your setup intact.

In [10]:
import torch.nn as nn
from deeplay import Layer, Config

block_template = Layer("layer") >> Layer("activation") >> Layer("normalization") >> Layer("pool")

config = (
    Config()
    .backbone.encoder_blocks(block_template)
    .backbone.encoder_blocks.normalization(nn.LazyBatchNorm2d)
)

**Note**: The string you pass into the `Layer` object directly corresponds to the name you'll use in the `Config` object. This is how Deeplay knows what layer you're trying to customize or add. For example, the `"normalization"` string in `Layer("normalization")` is what you'll use to set the normalization layer type in the config, as in `.backbone.blocks.normalization(nn.LazyBatchNorm2d)`.

**Note**: It's worth noting that `normalization` wasn't a default component in the original `ImageClassifier`. By adding* it to our custom `block_template`, we've effectively expanded the architecture of the model. After adding a new layer to a template, you should also specify its settings in the Config. Here, we're setting it to use `nn.LazyBatchNorm2d`.

### 3. Using Pretrained Torch Modules

**Incorporate Pretrained Models for a Quick Start**

Deeplay allows for seamless integration of pretrained PyTorch models, making it incredibly easy to leverage existing architectures. Take for example a pretrained ResNet-18 model from torchvision:

This offers a quick and straightforward way to benefit from well-established architectures while still maintaining the flexibility that Deeplay provides.

Is this in line with what you're looking for? Feel free to make edits or suggest revisions!

In [11]:
import torchvision

from deeplay import ImageClassifier, Config

backbone = torchvision.models.resnet18(pretrained=True)

# Remove the pooling and fully connected layers as they are not needed
backbone.avgpool = nn.Identity()
backbone.fc = nn.Identity()

classifier = ImageClassifier.from_config(
    Config()
    .num_classes(10)
    .backbone(backbone)
)

print(classifier)

ImageClassifier(
  (num_classes): 10
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentu

c:\Users\GU\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\GU\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)




Here, we directly use the pretrained ResNet-18 model as the `backbone` in our `ImageClassifier`. Note that you might need to remove or adapt certain layers, like we did with the average pooling (`avgpool`) and fully connected (`fc`) layers.


**Note**: Non-Deeplay modules cannot be further customized using Deeplay. 